In [1]:
import sys
import os
import numpy as np

SCRIPT_DIR = os.path.dirname(os.path.abspath("..."))
sys.path.append(os.path.dirname(SCRIPT_DIR))

from training.generate import (generate_response, load_model_tokenizer_for_generate, 
                               get_special_token_id, get_special_token_id)
from training.consts import END_KEY, PROMPT_FORMAT, RESPONSE_KEY_NL
from training.trainer import PROMPT_FORMAT, create_data_set_from_json_list
from datasets import load_from_disk
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
)

/opt/bo_ling/python39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load goldern dataset that is unseen in train

In [2]:

test_data = load_from_disk("/opt/home/bo_ling/dataset/gds_v4.hf")
test_data

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 182662
})

In [3]:
test_data[0]

{'instruction': 'Extract dob from the following input:',
 'input': 'California DRIVER LICENSE 081202 LN SOTO MUNGUIA FN ALEC PIERRE 9939 VAN RUITEN ST BELLFLOWER, CA 90706 SEX M AM DONOR USA DLY7679862 EXP 08/12/2025 DOB 08/12/2002 AGE 2 IN 7023 DD 08/04/2021606A3/E5FD/25 CLASS C END NONE RSTR NONE 08122002 HAIR BRN EYES HZL HGT 5\'-11" WGT 205 lb FEDERAL LIMITS APPLY ISS 10/14/2021',
 'output': '\n2002-08-12',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nExtract dob from the following input:\nCalifornia DRIVER LICENSE 081202 LN SOTO MUNGUIA FN ALEC PIERRE 9939 VAN RUITEN ST BELLFLOWER, CA 90706 SEX M AM DONOR USA DLY7679862 EXP 08/12/2025 DOB 08/12/2002 AGE 2 IN 7023 DD 08/04/2021606A3/E5FD/25 CLASS C END NONE RSTR NONE 08122002 HAIR BRN EYES HZL HGT 5\'-11" WGT 205 lb FEDERAL LIMITS APPLY ISS 10/14/2021\n\n### Response:\n2002-08-12\n'}

# Load checkpoint at step: 19600 (12% of total data in first epoch) 

In [4]:
local_output_dir="/opt/home/doc_transcript_pii_checkpoint-49200"
#model, tokenizer = load_model_tokenizer_for_generate(local_output_dir)
import torch
tokenizer = AutoTokenizer.from_pretrained(local_output_dir, padding_side="left")
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(local_output_dir, trust_remote_code=True).to(device)

In [5]:
def generate_helpdesk_response(
    instruction: str,
    input_text: str,
    model: PreTrainedModel,
    tokenizer: PreTrainedTokenizer,
    do_sample: bool = True,
    max_new_tokens: int = 256,
    top_p: float = 0.92,
    top_k: int = 0,
    **kwargs,
) -> str:
    texts = PROMPT_FORMAT.format(instruction=instruction, input_text=input_text, output_text="")
    input_ids = tokenizer(texts, return_tensors="pt").input_ids.to("cuda")

    response_key_token_id = get_special_token_id(tokenizer, RESPONSE_KEY_NL)
    end_key_token_id = get_special_token_id(tokenizer, END_KEY)

    gen_tokens = model.generate(
        input_ids,
        pad_token_id=tokenizer.pad_token_id,
        # Ensure generation stops once it generates "### End"
        eos_token_id=end_key_token_id,
        do_sample=do_sample,
        max_new_tokens=max_new_tokens,
        top_p=top_p,
        top_k=top_k,
        **kwargs,
    )[0].cpu()

    # The response will be set to this variable if we can identify it.
    decoded = None

    # Find where "### Response:" is first found in the generated tokens.  Considering this is part of the prompt,
    # we should definitely find it.  We will return the tokens found after this token.
    response_pos = None
    response_positions = np.where(gen_tokens == response_key_token_id)[0]
    if len(response_positions) == 0:
        logger.warn(f"Could not find response key {response_key_token_id} in: {gen_tokens}")
    else:
        response_pos = response_positions[0]

    if response_pos:
        # Next find where "### End" is located.  The model has been trained to end its responses with this sequence
        # (or actually, the token ID it maps to, since it is a special token).  We may not find this token, as the
        # response could be truncated.  If we don't find it then just return everything to the end.  Note that
        # even though we set eos_token_id, we still see the this token at the end.
        end_pos = None
        end_positions = np.where(gen_tokens == end_key_token_id)[0]
        if len(end_positions) > 0:
            end_pos = end_positions[0]

        decoded = tokenizer.decode(gen_tokens[response_pos + 1 : end_pos]).strip()

    return decoded

# Print some sample output for human to read

In [6]:
count = 0
for d in test_data:
    instruction = d["instruction"]
    input_text= d["input"]
    generated = generate_helpdesk_response(instruction, input_text, model, tokenizer)
    expected = d['output']
    print("="*100)
    print("="*100)
    print("INSTRUCTION:")
    print(instruction)
    print("\nINPUT:")
    print(input_text)
    print("\nGENERATED:")
    print(generated)
    print("\nEXPECTED:")
    print(expected)
    count += 1
    if count > 10:
        break

INSTRUCTION:
Extract dob from the following input:

INPUT:
California DRIVER LICENSE 081202 LN SOTO MUNGUIA FN ALEC PIERRE 9939 VAN RUITEN ST BELLFLOWER, CA 90706 SEX M AM DONOR USA DLY7679862 EXP 08/12/2025 DOB 08/12/2002 AGE 2 IN 7023 DD 08/04/2021606A3/E5FD/25 CLASS C END NONE RSTR NONE 08122002 HAIR BRN EYES HZL HGT 5'-11" WGT 205 lb FEDERAL LIMITS APPLY ISS 10/14/2021

GENERATED:
2002-08-12

EXPECTED:

2002-08-12
INSTRUCTION:
Extract issue date from the following input:

INPUT:
California DRIVER LICENSE 081202 LN SOTO MUNGUIA FN ALEC PIERRE 9939 VAN RUITEN ST BELLFLOWER, CA 90706 SEX M AM DONOR USA DLY7679862 EXP 08/12/2025 DOB 08/12/2002 AGE 2 IN 7023 DD 08/04/2021606A3/E5FD/25 CLASS C END NONE RSTR NONE 08122002 HAIR BRN EYES HZL HGT 5'-11" WGT 205 lb FEDERAL LIMITS APPLY ISS 10/14/2021

GENERATED:
2021-10-14

EXPECTED:

2021-10-14
INSTRUCTION:
Extract expiration date from the following input:

INPUT:
California DRIVER LICENSE 081202 LN SOTO MUNGUIA FN ALEC PIERRE 9939 VAN RUITE

In [7]:
def normalize_str(s):
    return s.replace("\n", "").strip().lower()
normalize_str("OSCAR") == normalize_str("\n\nOscar ")

True

#  Compute accuracy based on instruction on goldern dataset

In [8]:
count = 0
statistics = {}
for d in test_data:
    instruction = d["instruction"]
    input_text= d["input"]
    generated = generate_helpdesk_response(instruction, input_text, model, tokenizer)
    expected = d['output']
    if instruction in statistics:
        stat = statistics[instruction]
    else:
        stat = {"eq": 0, "neq": 0}
        statistics[instruction] = stat
    if normalize_str(generated) == normalize_str(expected):
        stat["eq"] += 1
    else:
        stat["neq"] += 1
    count += 1
    if count > 3000:
        break

In [9]:
print(f"******************** Performance for doc transcript pii *****************\n")
for k ,v in statistics.items():
    v["accuracy"] = v["eq"] / (v["eq"] + v['neq'])
    print(f"=== The accuracy for task `{k}`: {v['accuracy']} on {(v['eq'] + v['neq'])}  samples ===")

******************** Performance for doc transcript pii *****************

=== The accuracy for task `Extract dob from the following input:`: 0.9641255605381166 on 223  samples ===
=== The accuracy for task `Extract issue date from the following input:`: 0.8430493273542601 on 223  samples ===
=== The accuracy for task `Extract expiration date from the following input:`: 0.9865470852017937 on 223  samples ===
=== The accuracy for task `Extract first name from the following input:`: 0.8340807174887892 on 223  samples ===
=== The accuracy for task `Extract middle name from the following input:`: 0.8342245989304813 on 187  samples ===
=== The accuracy for task `Extract last name from the following input:`: 0.6771300448430493 on 223  samples ===
=== The accuracy for task `Extract license class from the following input:`: 0.9417040358744395 on 223  samples ===
=== The accuracy for task `Extract drivers license number from the following input:`: 0.7130044843049327 on 223  samples ===
=== The 

# Accuracy on test dataset that is unseen

In [14]:
print(f"******************** Performance for doc transcript pii *****************\n")
for k ,v in statistics.items():
    v["accuracy"] = v["eq"] / (v["eq"] + v['neq'])
    print(f"=== The accuracy for task `{k}`: {v['accuracy']} on {(v['eq'] + v['neq'])}  samples ===")

******************** Performance for doc transcript pii *****************

=== The accuracy for task `Extract first name from the following input:`: 0.7246376811594203 on 69  samples ===
=== The accuracy for task `Extract zip from the following input:`: 0.9852941176470589 on 68  samples ===
=== The accuracy for task `Extract issue date from the following input:`: 0.9710144927536232 on 69  samples ===
=== The accuracy for task `Extract license class from the following input:`: 0.8857142857142857 on 70  samples ===
=== The accuracy for task `Extract last name from the following input:`: 0.609375 on 64  samples ===
=== The accuracy for task `Extract drivers license number from the following input:`: 0.7647058823529411 on 68  samples ===
=== The accuracy for task `Extract middle name from the following input:`: 0.7945205479452054 on 73  samples ===
=== The accuracy for task `Extract address from the following input:`: 0.7261904761904762 on 84  samples ===
=== The accuracy for task `Extract

## Accuracy when load earlier checkpoint 19600 (performance is worse)

In [31]:
print(f"******************** Performance for doc transcript pii *****************\n")
for k ,v in statistics.items():
    v["accuracy"] = v["eq"] / (v["eq"] + v['neq'])
    print(f"======= The accuracy for task `{k}`: {v['accuracy']}  ========")

******************** Performance for doc transcript pii *****************

======= The accuracy for task `Extract first name from the following input:`: 0.6363636363636364  ========
======= The accuracy for task `Extract zip from the following input:`: 0.9166666666666666  ========
======= The accuracy for task `Extract issue date from the following input:`: 1.0  ========
======= The accuracy for task `Extract license class from the following input:`: 1.0  ========
======= The accuracy for task `Extract last name from the following input:`: 0.4444444444444444  ========
======= The accuracy for task `Extract drivers license number from the following input:`: 0.7058823529411765  ========
======= The accuracy for task `Extract middle name from the following input:`: 0.775  ========
======= The accuracy for task `Extract address from the following input:`: 0.8461538461538461  ========
======= The accuracy for task `Extract driving licence issue state from the following input:`: 0.9459459459